In [1]:
import datetime
import numpy as np
import os
from PIL import Image

In [2]:
needle_model_name = "NeedleModel"
sequence_browser_name = "NeedleCapture_Tofu_02"
image_name = "Image_Image"

output_folder = r"NeedleCapture\2020-03-16-NeedleCapture-Medium"
filename_prefix = "NeedleCapture_NoNeedle_Tofu_02"

num_needle_slices = 5

In [3]:
browser_node = slicer.util.getFirstNodeByName(sequence_browser_name, className='vtkMRMLSequenceBrowserNode')
model_node = slicer.util.getFirstNodeByName(needle_model_name, className="vtkMRMLModelNode")
image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")

if browser_node is None:
    logging.error("Could not find input browser node: {}".format(sequence_browser_name))
    raise

if model_node is None:
    logging.error("Could not find input model node: {}".format(needle_model_name))
    raise

if image_node is None:
    logging.error("Could not find input image node: {}".format(image_name))
    raise

In [4]:
labelmap_name = "temp_labelmap"
labelmap_node = slicer.util.getFirstNodeByName(labelmap_name, className="vtkMRMLLabelMapVolumeNode")
if labelmap_node is None:
    labelmap_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelmap_node.SetName(labelmap_name)

In [5]:
segmentation_name = "Segmentation"
segmentation_node = slicer.util.getFirstNodeByName(segmentation_name, className="vtkMRMLSegmentationNode")
if segmentation_node is None:
    segmentation_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    segmentation_node.SetName(segmentation_name)

In [6]:
image_copy_name = "Image_Image_3DEmpty"
image_copy_node = slicer.util.getFirstNodeByName(image_copy_name, className="vtkMRMLScalarVolumeNode")
if image_copy_node is None:
    image_copy_node = slicer.mrmlScene.CopyNode(image_node)
    image_copy_node.SetName(image_copy_name)

In [7]:
z_extent = [-1*(num_needle_slices//2), num_needle_slices//2]
print(z_extent)

[-2, 2]


In [8]:
#get current image extent and spacing
image_extent = image_copy_node.GetImageData().GetExtent()
image_spacing = image_copy_node.GetSpacing()

#change extent of image (make thicker in z direction) and allocate necessary space
image_copy_node.GetImageData().SetExtent(image_extent[0], image_extent[1], image_extent[2], image_extent[3], z_extent[0], z_extent[1])
image_copy_node.GetImageData().AllocateScalars(image_copy_node.GetImageData().GetScalarType(), image_copy_node.GetImageData().GetNumberOfScalarComponents())

#shift so image extent starts at 0
image_copy_node.ShiftImageDataExtentToZeroStart()
print("Original Extent: {}\nModified Extent: {}".format(image_extent, image_copy_node.GetImageData().GetExtent()))

#increase image spacing
image_copy_node.SetSpacing(image_spacing[0], image_spacing[1], 2.0)
print("Original Spacing: {}\nModified Spacing: {}".format(image_spacing, image_copy_node.GetSpacing()))

Original Extent: (0, 511, 0, 511, 0, 0)
Modified Extent: (0, 511, 0, 511, 0, 4)
Original Spacing: (1.0, 1.0, 1.0)
Modified Spacing: (1.0, 1.0, 2.0)


In [9]:
import vtkSegmentationCorePython as vtkSegmentationCore

def removeSegments(selectedSegmentation):
    num_segments = selectedSegmentation.GetSegmentation().GetNumberOfSegments()
    for i in range(num_segments):
        segmentId = selectedSegmentation.GetSegmentation().GetNthSegmentID(i)
        selectedSegmentation.RemoveSegment(segmentId)

In [10]:
browser_node.SelectLastItem()

segmentation_logic = slicer.modules.segmentations.logic()

notAllEmpty=False

image_cast = vtk.vtkImageCast()
image_cast.SetOutputScalarTypeToUnsignedChar()
image_cast.Update()

removeSegments(segmentation_node)
n = browser_node.GetNumberOfItems()

all_seg_files = np.zeros((n, 512, 512, num_needle_slices))
all_us_files = np.zeros((n, 512, 512, num_needle_slices))
counter = 0

for i in range(n):
    
    #Get segmented image in US position
    segmentation_logic.ImportModelToSegmentationNode(model_node, segmentation_node)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_node, labelmap_node, image_copy_node)
    segmented_image = labelmap_node.GetImageData()
    
    image_cast.SetInputData(segmented_image)
    image_cast.Update()
    
    #Set ultrasound image filename
    img_file_name = filename_prefix + "_%04d_ultrasound" % i
    ultrasound_fullname = os.path.join(output_folder, img_file_name)
    
    #Set segmentation image filenames
    seg_file_name = filename_prefix + "_%04d_segmentation" % i
    segmentation_fullname = os.path.join(output_folder, seg_file_name)
    
    #Set combined image filename
    combined_file_name = filename_prefix + "_%04d_combined" % i
    combined_fullname = os.path.join(output_folder, combined_file_name)
    
    #Get ultrasound image from Slicer
    ultrasound_array = slicer.util.arrayFromVolume(image_node)
    
    #Expand dimension of US by "smearing" so dimensionality matches number of slices from needle volume
    ultrasound_array = ultrasound_array[0,:,:]
    ultrasound_array = np.repeat(ultrasound_array[:, :, np.newaxis], num_needle_slices, axis=2)
    
    #Get slices from needle volume from slicer
    segmentation_array_temp = slicer.util.arrayFromVolume(labelmap_node)
    
    segmentation_array = np.zeros((512,512,num_needle_slices))
    
    # way to ensure you're getting slices from "middle" of volume
    # this needs to be improved and/or moved to before extent is set
    volume_slices = segmentation_array.shape[0]
    middle_ix = volume_slices // 2
    curr_slice = 0
    for ix in range(num_needle_slices): #+1):
        segmentation_array[:,:,ix] = segmentation_array_temp[ix,:,:]
        curr_slice = curr_slice + 1
        
    #OPTIONAL
    #If needle is visible in segmentation
    #if np.amax(segmentation_array) > 0: #and i % 2 ==0:
         
    #save current US and needle arrays
    all_seg_files[counter,:,:,:] = segmentation_array
    all_us_files[counter,:,:,:] = ultrasound_array
    counter = counter + 1
    
    #OPTIONAL
    #export combined images as jpeg - can then be used in pix2pix non volume
    #combined_array = np.concatenate((segmentation_array,ultrasound_array),axis=1)
    #im = Image.fromarray(combined_array[:,:,0]).convert('RGB')
    #im.save(combined_fullname + ".jpeg")
    
    #Delete old segmentations
    removeSegments(segmentation_node)
    
    slicer.app.processEvents()
    browser_node.SelectNextItem()
    
array_filename = filename_prefix + "_array_" + str(num_needle_slices) + "d.npz";
print(all_seg_files.shape)
print("Number of files exported: {}".format(counter))

#Save all US and needle images as a zipped numpy array
np.savez_compressed(os.path.join(output_folder, array_filename), all_seg_files[:counter,:,:,:], all_us_files[:counter,:,:,:])

(655, 512, 512, 5)
Number of files exported: 655
